In [ ]:
import os
import pickle
import tsplib95
import csv
import time
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [ ]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from pkl {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G.to_undirected()

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    # print(f"Loading {name} from tsplib {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    # remove redundant edges
    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    # remove self loops
    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    # save as pkl file
    with open(graph_file_path, "wb") as file:
        print(f"Saving {name} to pkl {graph_file_path}")
        pickle.dump(G, file)

    return G

In [ ]:
tsp_dataset_file_path = os.path.join("tsp_dataset", "tsp_dataset.csv")
print(f"Loading tsp dataset from {tsp_dataset_file_path}")
tsp_dataset = pd.read_csv(tsp_dataset_file_path)

In [ ]:
def get_opt_tour_length(name):
    return tsp_dataset[tsp_dataset["name"] == name]["opt_tour_length"].values[0]

In [ ]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [ ]:
def get_apx_ratio(G: nx.graph, tour: list):
    return get_tour_length(G, tour) / get_opt_tour_length(G.name)

In [ ]:
tsp_dataset

In [ ]:
from mst import MST
st = MST()

In [ ]:
def get_total_edge_weight(G: nx.graph):
    total_edge_weight = 0
    for u, v in G.edges:
        total_edge_weight += G[u][v]["weight"]
    return total_edge_weight

In [ ]:
data = []

In [162]:
with open("christofides_results.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    for name in tsp_dataset["name"]:
        print(name)

        G = get_tsp_graph(name)
        dimension = G.number_of_nodes()
        opt_tour_length = get_opt_tour_length(name)

        d = {
            "name": name,
            "dimension": dimension,
            "opt_tour_length": opt_tour_length,
        }

        start_time = time.time()

        tree = st.get_mst_k(G, 1)
        tour = christofides(G, tree=tree)
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        tour_length = get_tour_length(G, tour)
        apx_ratio = tour_length / opt_tour_length

        d["tour_length"] = tour_length
        d["apx_ratio"] = apx_ratio
        d["elapsed_time"] = elapsed_time

        print(list(d.values()))
        writer.writerow(d.values())

burma14
['burma14', 14, 3323, 3559, 1.0710201625037616, 0.011527299880981445]
ulysses16
['ulysses16', 16, 6859, 6982, 1.0179326432424551, 0.014864444732666016]
gr17
['gr17', 17, 2085, 2211, 1.0604316546762589, 0.0004448890686035156]
gr21
['gr21', 21, 2707, 2879, 1.0635389730328777, 0.01566624641418457]
ulysses22
['ulysses22', 22, 7013, 7382, 1.0526165692285756, 0.036273956298828125]
gr24
['gr24', 24, 1272, 1482, 1.1650943396226414, 0.011542558670043945]
fri26
['fri26', 26, 937, 988, 1.0544290288153682, 0.021932601928710938]
bayg29
['bayg29', 29, 1610, 1749, 1.0863354037267081, 0.002742290496826172]
bays29
['bays29', 29, 2020, 2188, 1.0831683168316832, 0.013531208038330078]
swiss42
['swiss42', 42, 1273, 1399, 1.09897879025923, 0.022762537002563477]
dantzig42
['dantzig42', 42, 699, 792, 1.1330472103004292, 0.047570228576660156]
gr48
['gr48', 48, 5046, 5684, 1.1264367816091954, 0.04155898094177246]
hk48
['hk48', 48, 11461, 13059, 1.1394293691649944, 0.05800938606262207]
eil51
['eil51', 51